In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go

import dash
from dash import html, dcc, Input, Output

from sqlalchemy import create_engine
import snowflake.connector

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.1), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


In [2]:
import snowflake.connector

In [3]:
## Set options to view all columns
pd.set_option('display.max_columns', None)

In [6]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'RADAR_CHARTS'

In [7]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)

In [8]:
cursor = conn.cursor()

In [14]:
cursor.execute('SELECT * FROM STANDARD_RADAR')
standad_chart_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
standard_chart_data = pd.DataFrame(standad_chart_rows, columns=column_names)

In [15]:
cursor.execute('SELECT * FROM ATTACKING_RADAR')
attacking_chart_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
attacking_chart_data = pd.DataFrame(attacking_chart_rows, columns=column_names)

In [16]:
cursor.execute('SELECT * FROM DEFENDING_RADAR')
defending_chart_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
defending_chart_data = pd.DataFrame(defending_chart_rows, columns=column_names)

In [17]:
# standard_chart_data = pd.read_csv(r"C:\Users\ksbha\Documents\Python Scripts\Data Engineering Scraping project\Shiny Data\Standard_radar_chart_v2.csv")

In [18]:
# attacking_chart_data = pd.read_csv(r"C:\Users\ksbha\Documents\Python Scripts\Data Engineering Scraping project\Shiny Data\Attacking_radar_chart_v2.csv")

In [19]:
# defending_chart_data = pd.read_csv(r"C:\Users\ksbha\Documents\Python Scripts\Data Engineering Scraping project\Shiny Data\Defending_radar_chart_v2.csv")

In [20]:
team_data = standard_chart_data[standard_chart_data['TEAM_NAME'] == 'Brentford']

In [28]:
team_data.head()

,SEASON,TEAM_NAME,VARIABLE,NORM_VALUE,VALUE
32,2223,Brentford,Tackles Won (%),0.430906,0.574106
33,2223,Brentford,NPxG/Game,0.437500,1.330000
34,2223,Brentford,Goals/Game,0.428571,1.526316
35,2223,Brentford,Shots/Game,0.168675,10.526316
36,2223,Brentford,Conceded/Game,0.711111,1.210526


In [25]:
average_data = standard_chart_data[standard_chart_data['TEAM_NAME'] == 'Average_2223']
average_data[average_data['SEASON'] == 2223]

,SEASON,TEAM_NAME,VARIABLE,NORM_VALUE,VALUE
16,2223,Average_2223,Tackles Won (%),0.530159,0.583351
17,2223,Average_2223,Conceded/Game,0.528889,1.426316
18,2223,Average_2223,NPxG/Game,0.428125,1.321000
19,2223,Average_2223,Goals/Game,0.368254,1.426316
20,2223,Average_2223,Shots On Target (%),0.553671,0.330860
21,2223,Average_2223,Shots/Game,0.471888,12.513158
22,2223,Average_2223,xG Against/Game,0.378804,1.421500
23,2223,Average_2223,Pass Completion (%),0.490848,0.785704


In [40]:
def create_radar_chart(season, team_name, data, chart_name):
    team_data = data[data['TEAM_NAME'] == team_name]
    average_data = data[data['TEAM_NAME'] == 'Average_2223']

    team_data = team_data[team_data['SEASON'] == season]
    average_data = average_data[average_data['SEASON'] == season]

    # Prepare data for plotting
    categories = team_data['VARIABLE']
    norm_values = team_data['NORM_VALUE']
    average_norm_values = average_data['NORM_VALUE']
    values = team_data['VALUE']

    difference = [golden - average for golden, average in zip(norm_values, average_norm_values)]

    hover_text = (
    "<span style='font-size: 20px; color: #d3d3d3;'>%{theta}</span><br>"
    "<span style='font-size: 20px; color: white;'>Value: %{customdata[0]:.2f}</span><br>"
    "<span style='font-size: 15px; color: #d3d3d3;'>Difference from Average: %{customdata[1]:.2f}</span><extra></extra>"
    )

    # Create radar chart
    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=average_norm_values,
        theta=categories,
        name='AVERAGE',
        fillcolor='rgba(100, 100, 100, 0.65)',  # Different color for distinction
        line_color='rgba(100, 100, 100, 1)', # Line colour for Average plot
        fill='toself',
        customdata=np.stack((values, difference), axis=-1),
        hovertemplate=hover_text,
        marker=dict(
            size=1  # Hides the markers by setting their size to zero
        )
    ))

    fig.add_trace(go.Scatterpolar(
        r=norm_values,
        theta=categories,
        name=team_name,
        opacity=0.6,
        fillcolor='rgba(210, 210, 0, 0.6)',  # Adjusted for lighter opaque fill
        line_color='rgba(210, 210, 0, 1)',  # Adjusted for lighter line color
        fill='toself',
        customdata=np.stack((values, difference), axis=-1),
        hovertemplate=hover_text,
        marker=dict(
            size=1  # Hides the markers by setting their size to zero
        )
    ))

    fig.add_layout_image(
        dict(
            source='https://i.imgur.com/9yKFcv4.png',
            xref="paper", yref="paper",
            x=0.16, y=1,
            sizex=1.05, sizey=1.05,
            opacity=0.7,  # Adjust opacity as needed
            layer="below"
        )
    )

    for i, (value, category) in enumerate(zip(values, categories)):
        angle = (i / float(len(categories))) * 2 * np.pi 
        x = 0.5 + (1.1) * np.cos(angle) / 2
        y = 0.48 + (1.1) * np.sin(angle) / 1.8

        annotation_text = \
        f"<span style='font-size: 15px;'><b>{category}</b></span><br>" \
        f"<span style='line-height: 2px;'>&nbsp;</span><br>" \
        f"<span style='font-size: 20px; color: rgba(210, 210, 0, 1);'>{value:.2f}</span>"

        fig.add_annotation(
            x=x,
            y=y,
            xref="paper",
            yref="paper",
            text=annotation_text,  # Bold category name and value
            showarrow=False,
            font=dict(size=10, color='white'),
            align="center",
            xanchor='center',
            yanchor='middle',
        )

    # Update layout
    fig.update_layout(
        autosize=False,
        width=800,  # Set the width
        height=600,  # Set the height
        polar=dict(
            bgcolor='rgba(0,0,0,0)',
            radialaxis=dict(
                visible=False,
                range=[0, 1],
                linecolor='rgba(17, 17, 17, 1)',
                showline=False,
                gridcolor='white'
            ),
            angularaxis=dict(
                showline=False,  # Hide angular axis line
                gridcolor='rgba(0,0,0,0)',
                showticklabels=False  
            )
        ),
        paper_bgcolor='rgb(70, 70, 70)',      
        showlegend=False,
        title=f'{chart_name} for {team_name}',
        hoverlabel=dict(
            bgcolor="rgba(20, 20, 20, 0.8)",
            font_family="Roboto, sans-serif",
            bordercolor="rgba(20, 20, 20, 0.8)",),
        font=dict(
            family="Roboto, sans-serif",  # Specify the font family
            size=15,                     # Specify the font size
            color="white"                # Specify the font color
        )
    )

    return fig



In [41]:
fig = create_radar_chart(2223, 'Manchester City', defending_chart_data, "Defending Radar Chart")
fig.show()

In [42]:
fig = create_radar_chart(2223, 'Manchester City', attacking_chart_data, "Defending Radar Chart")
fig.show()

In [34]:
def standard_radar_chart(team_name, data):
    # Filter data for the selected team
    team_data = data[data['TEAM_NAME'] == team_name]
    average_data = data[data['TEAM_NAME'] == 'Average_2223']

    # Prepare data for plotting
    categories = team_data['VARIABLE']
    norm_values = team_data['NORM_VALUE']
    average_norm_values = average_data['NORM_VALUE']
    values = team_data['VALUE']
    

    difference = [golden - average for golden, average in zip(norm_values, average_norm_values)]

    hover_text = (
    "<span style='font-size: 20px; color: #d3d3d3;'>%{theta}</span><br>"
    "<span style='font-size: 20px; color: white;'>Value: %{customdata[0]:.2f}</span><br>"
    "<span style='font-size: 15px; color: #d3d3d3;'>Difference from Average: %{customdata[1]:.2f}</span><extra></extra>"
    )

    # Create radar chart
    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=average_norm_values,
        theta=categories,
        name='AVERAGE',
        fillcolor='rgba(100, 100, 100, 0.65)',  # Different color for distinction
        line_color='rgba(100, 100, 100, 1)', # Line colour for Average plot
        fill='toself',
        customdata=np.stack((values, difference), axis=-1),
        hovertemplate=hover_text,
        marker=dict(
            size=1  # Hides the markers by setting their size to zero
        )
    ))

    fig.add_trace(go.Scatterpolar(
        r=norm_values,
        theta=categories,
        name=team_name,
        opacity=0.6,
        fillcolor='rgba(210, 210, 0, 0.6)',  # Adjusted for lighter opaque fill
        line_color='rgba(210, 210, 0, 1)',  # Adjusted for lighter line color
        fill='toself',
        customdata=np.stack((values, difference), axis=-1),
        hovertemplate=hover_text,
        marker=dict(
            size=1  # Hides the markers by setting their size to zero
        )
    ))

    fig.add_layout_image(
        dict(
            source='https://i.imgur.com/9yKFcv4.png',
            xref="paper", yref="paper",
            x=0.16, y=1,
            sizex=1.05, sizey=1.05,
            opacity=0.7,  # Adjust opacity as needed
            layer="below"
        )
    )

    for i, (value, category) in enumerate(zip(values, categories)):
    # Calculate the angle for the category
        angle = (i / float(len(categories))) * 2 * np.pi 
        x = 0.5 + (1.1) * np.cos(angle) / 2
        y = 0.48 + (1.1) * np.sin(angle) / 1.8

        annotation_text = \
        f"<span style='font-size: 15px;'><b>{category}</b></span><br>" \
        f"<span style='line-height: 2px;'>&nbsp;</span><br>" \
        f"<span style='font-size: 20px; color: rgba(210, 210, 0, 1);'>{value:.2f}</span>"

        fig.add_annotation(
            x=x,
            y=y,
            xref="paper",
            yref="paper",
            text=annotation_text,  # Bold category name and value
            showarrow=False,
            font=dict(size=10, color='white'),
            align="center",
            # The xanchor and yanchor are set to 'middle' and 'center' to center the annotations
            xanchor='center',
            yanchor='middle',
        )

    # Update layout
    fig.update_layout(
        autosize=False,
        width=800,  # Set the width
        height=600,  # Set the height
        polar=dict(
            bgcolor='rgba(0,0,0,0)',
            radialaxis=dict(
                visible=False,
                range=[0, 1],
                linecolor='rgba(17, 17, 17, 1)',
                showline=False,
                gridcolor='white'
            ),
            angularaxis=dict(
                showline=False,  # Hide angular axis line
                gridcolor='rgba(0,0,0,0)',
                showticklabels=False  
            )
        ),
        paper_bgcolor='rgb(70, 70, 70)',      
        showlegend=False,
        title=f'Radar Chart for {team_name}',
        hoverlabel=dict(
            bgcolor="rgba(20, 20, 20, 0.8)",
            font_family="Roboto, sans-serif",
            bordercolor="rgba(20, 20, 20, 0.8)",),
        font=dict(
            family="Roboto, sans-serif",  # Specify the font family
            size=15,                     # Specify the font size
            color="white"                # Specify the font color
        )
    )

    return fig

# Create and show the radar chart for a specific team
fig = standard_radar_chart('Manchester City', standard_chart_data)
fig.show()

In [35]:
def create_radar_chart(season, team_name, data, chart_name):
    team_data = data[data['TEAM_NAME'] == team_name]
    average_data = data[data['TEAM_NAME'] == 'Average_2223']

    team_data = team_data[team_data['SEASON'] == season]
    average_data = average_data[average_data['SEASON'] == season]

    # Prepare data for plotting
    categories = team_data['VARIABLE']
    norm_values = team_data['NORM_VALUE']
    average_norm_values = average_data['NORM_VALUE']
    values = team_data['VALUE']

    difference = [golden - average for golden, average in zip(norm_values, average_norm_values)]

    hover_text = (
    "<span style='font-size: 20px; color: #d3d3d3;'>%{theta}</span><br>"
    "<span style='font-size: 20px; color: white;'>Value: %{customdata[0]:.2f}</span><br>"
    "<span style='font-size: 15px; color: #d3d3d3;'>Difference from Average: %{customdata[1]:.2f}</span><extra></extra>"
    )

    # Create radar chart
    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=average_norm_values,
        theta=categories,
        name='AVERAGE',
        fillcolor='rgba(100, 100, 100, 0.65)',  # Different color for distinction
        line_color='rgba(100, 100, 100, 1)', # Line colour for Average plot
        fill='toself',
        customdata=np.stack((values, difference), axis=-1),
        hovertemplate=hover_text,
        marker=dict(
            size=1  # Hides the markers by setting their size to zero
        )
    ))

    fig.add_trace(go.Scatterpolar(
        r=norm_values,
        theta=categories,
        name=team_name,
        opacity=0.6,
        fillcolor='rgba(210, 210, 0, 0.6)',  # Adjusted for lighter opaque fill
        line_color='rgba(210, 210, 0, 1)',  # Adjusted for lighter line color
        fill='toself',
        customdata=np.stack((values, difference), axis=-1),
        hovertemplate=hover_text,
        marker=dict(
            size=1  # Hides the markers by setting their size to zero
        )
    ))

    fig.add_layout_image(
        dict(
            source='https://i.imgur.com/9yKFcv4.png',
            xref="paper", yref="paper",
            x=0.296, y=1.01,
            sizex=0.405, sizey=1.05,
            opacity=0.7,  # Adjust opacity as needed
            layer="below",
            sizing="stretch"
        )
    )

    for i, (value, category) in enumerate(zip(values, categories)):
        angle = (i / float(len(categories))) * 2 * np.pi 
        x = 0.5 + (1.1) * np.cos(angle) / 4
        y = 0.48 + (1.1) * np.sin(angle) / 1.65

        annotation_text = \
        f"<span style='font-size: 10px;'><b>{category}</b></span><br>" \
        f"<span style='font-size: 13px; color: rgba(210, 210, 0, 1);'>{value:.2f}</span>"

        fig.add_annotation(
            x=x,
            y=y,
            xref="paper",
            yref="paper",
            text=annotation_text,  # Bold category name and value
            showarrow=False,
            font=dict(size=10, color='white'),
            align="center",
            xanchor='center',
            yanchor='middle',
        )

    # Update layout
    fig.update_layout(
        autosize=True,
        width=355,  # Set the width
        height=300,  # Set the height
        polar=dict(
            bgcolor='rgba(0,0,0,0)',
            radialaxis=dict(
                visible=False,
                range=[0, 1],
                linecolor='rgba(17, 17, 17, 1)',
                showline=False,
                gridcolor='white'
            ),
            angularaxis=dict(
                showline=False,  # Hide angular axis line
                gridcolor='rgba(0,0,0,0)',
                showticklabels=False  
            )
        ),
        paper_bgcolor='rgb(70, 70, 70)',      
        showlegend=False,
        title={
            'text': f'{chart_name} for {team_name}',
            'y':0.95,  # Sets the y position of the title (1 is the top of the figure)
            'x':0.5,  # Centers the title horizontally (0.5 is the center of the figure)
            'xanchor': 'center',  # Ensures the title is centered at the x position
            'yanchor': 'top'  # Ensures the title is at the top of the y position
        },
        hoverlabel=dict(
            bgcolor="rgba(20, 20, 20, 0.8)",
            font_family="Roboto, sans-serif",
            bordercolor="rgba(20, 20, 20, 0.8)",),
        font=dict(
            family="Roboto, sans-serif",  # Specify the font family
            size=15,                     # Specify the font size
            color="white"                # Specify the font color
        )
    )

    return fig

In [36]:
# st.title('Team Radar Charts')
# team_name = st.selectbox('Select a Team', list(sorted(np.delete(standard_chart_data['TEAM_NAME'].unique(), 
#                                                          np.where(standard_chart_data['TEAM_NAME'].unique() == 'Average_2223')))))
# season = st.selectbox('Select a Season', (standard_chart_data['SEASON'].unique()))

# col1, col2 = st.columns(2)

# with col1:
#     st.header('Standard Radar Chart')
#     fig_standard = create_radar_chart(season, "Manchester City", standard_chart_data, "Standard Radar Chart")
#     st.plotly_chart(fig_standard, use_container_width=False)

# with col2:
#     st.header('Attacking Radar Chart')
#     fig_attacking = create_radar_chart(season, "Manchester City", attacking_chart_data, "Attacking Radar Chart") # Assuming the same data is used
#     st.plotly_chart(fig_attacking, use_container_width=False)

2023-12-14 12:09:49.337 
  command:

    streamlit run d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [38]:
fig_standard = create_radar_chart(2223, "Manchester City", standard_chart_data, "Standard Radar Chart")
fig_standard.show()